<a href="https://colab.research.google.com/github/dannyljw/new_bicycle/blob/main/%EC%A0%95%EB%A6%AC_%EC%99%84%EB%A3%8C_%EC%BD%94%EB%93%9C%EC%A0%9C%EC%B6%9C_v1_0_220704.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# feature 추가
> 체감온도, 불쾌지수, 네이버 날씨에서 보는 것과 같이 미세먼지 농도에 따른 나쁨/좋음/보통 값 부여, 
> 
> rental 되는 자전거 수 비율 적용, date 칼럼을 월/연/일로 나누기
> 
> train/test 데이터에 따라서 각기 다르게 요일 부여

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import math
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import sklearn
from sklearn.preprocessing import *
from sklearn.preprocessing import QuantileTransformer

# 체감온도 공식
def temp_sensible(df):
    #체감온도 공식은 체감온도(℃)=13.12+0.6215×T-11.37V0.16+0.3965V0.16×T
    temp_df_t=df['temp_mean'].tolist()
    temp_df_w=df['wind_mean'].tolist()
    temp_df_t
    temp_df_w
    result_=[]
    for i in range(len(temp_df_t)):
        result=13.12+0.6215*temp_df_t[i]-11.37*math.pow(temp_df_w[i],0.16)+0.3965*math.pow(temp_df_w[i],0.16)*temp_df_t[i]
        result_.append(round(result,1))
    df.insert(4,'temp_sensible',result_)

# 불쾌지수 공식
def discomfort_index(df):
    #0.81 * temp_df_t[i] +0.01 * temp_df_t[i]*((0.99*temp_df_t[i]) - 14.3)+ 46.3
    #불쾌지수 공식(80이상 : 매우 불쾌, 80~75: 50%불쾌 , 68~75 :불쾌감 시작, 68미만: 쾌적함)
    #discomfort_index(불쾌지수)
    temp_df_t=df['temp_mean'].tolist()
    temp_df_h=df['humidity'].tolist()
    temp_df_t
    temp_df_h
    result_=[]
    for i in range(len(temp_df_t)):
        result=(9/5*temp_df_t[i])-0.55*(1-temp_df_h[i]/100)*(9/5*temp_df_t[i]-26)+32
        result_.append(round(result,1))
    result_
    df['discomfort_index']=result_

# 미세먼지 PM10, PM2.5 에 따라서 값을 (class) 를 부여
def pm_trafficlight(df):
    #미세먼지 신호등 
    pm10=df['PM10'].tolist()
    pm2=df['PM2.5'].tolist()
    pm_=[]
    for i in range(len(pm10)):
        if(pm10[i]<=30 or pm2[i]<=15):
            pm_.append(1)
        elif((pm10[i]<=80 and pm10[i]>30) or (pm2[i] >=16 and pm2[i]<=50)):
            pm_.append(2)
        elif((pm10[i]<=150 and pm10[i]>80)or (pm2[i]>50 and pm2[i]<=100)):
            pm_.append(3)
        elif(pm10[i]>=150 or (pm2[i]>100)):
            pm_.append(4)
    df['pm']=pm_

# 연도별로 rental 값이 전체적으로 상향됨을 확인하여 비율 맞춰줌
# 비슷한 맥락으로, 전처리 이후, 예측 prediction 을 진행 한 뒤에 1.35를 곱함
def rental_rate(df):
    #연도별 증가하는 비율을 조사하고 전체적인 비율을 맞춰주기 위해서 값들을 변환해주었다.--> 추가 설명 필요
    y1 = df[df['year'] == 2018]['rental'] * 2.3
    y2 = df[df['year'] == 2019]['rental'] * 1.2
    y3 = df[df['year'] == 2020]['rental']
    new = pd.concat([y1, y2, y3], axis=0).to_frame()
    df['rental'] = new['rental']
    return True

# date 에서 연도 칼럼, 월 칼럼, 일 칼럼 따로 분리 
def seperate_date(df):
    date=df['date'].tolist()
    year=[]
    month=[]
    day=[]
    year_=''
    month_=''
    day_=''
    for j in date:
        list_=j.split("-")
        for i in range(3):
            if(i==0):
                year_+=list_[i]
            elif(i==1):
                month_+=list_[i]
            elif(i==2):
                day_+=list_[i]
        year.append(int(year_))
        month.append(int(month_))
        day.append(int(day_))
        year_=''
        month_=''
        day_=''
    df=df.drop(columns='date')
    df['year']=year
    df['month']=month
    df['day']=day
    return df

# train 데이터에서 요일 칼럼을 따로 추출하는 작업
# test 데이터와 달리 하는 이유는 시작하는 요일이 다르기 때문.
def train_day_of_the_week(df):
    day = []
    for i in range(0,614):
        if df.index[i] % 7 == 0:
            day.append("Monday")
        elif df.index[i] % 7 == 1:
            day.append("Tuesday")
        elif df.index[i] % 7 ==2:
            day.append("Wednesday")
        elif df.index[i] % 7 == 3:
            day.append("Thursday")
        elif df.index[i] % 7 == 4:
            day.append("Friday")
        elif df.index[i] % 7 == 5:
            day.append("Saturday")
        elif df.index[i] % 7 == 6:
            day.append("Sunday")
    for i in range(0,len(df.day)-614):
        if df.index[i] % 7 == 0:
            day.append("Sunday")
        elif df.index[i] % 7 == 1:
            day.append("Monday")
        elif df.index[i] % 7 == 2:
            day.append("Tuesday")
        elif df.index[i] % 7 == 3:
            day.append("Wednesday")
        elif df.index[i] % 7 == 4:
            day.append("Thursday")
        elif df.index[i] % 7 == 5:
            day.append("Friday")
        elif df.index[i] % 7 == 6:
            day.append("Saturday")
    df["day_name"] = day
    return df

# test 데이터 에서 요일 칼럼을 따로 만드는 함수
def test_day_of_the_week(df):
    day2= []
    for i in range(0,365):
        if df.index[i] % 7 == 0:
            day2.append("Friday")
        elif df.index[i] % 7 == 1:
            day2.append("Saturday")
        elif df.index[i] % 7 ==2:
            day2.append("Sunday")
        elif df.index[i] % 7 == 3:
            day2.append("Monday")
        elif df.index[i] % 7 == 4:
            day2.append("Tuesday")
        elif df.index[i] % 7 == 5:
            day2.append("Wednesday")
        elif df.index[i] % 7 == 6:
            day2.append("Thursday")
    df["day_name"] = day2  
    return df

# 데이터 불러오기

In [ ]:
sample = pd.read_csv('sample_submission.csv')
test_df = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [ ]:
train.head()

,date,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,rental
0,2018-01-01,NaN,-1.3,3.8,-5.1,34.0,17.0,39.1,8.3,86.5,1.4,3.8,4950
1,2018-01-02,NaN,-1.8,1.8,-4.3,36.0,22.0,42.0,7.9,82.3,1.8,4.9,7136
2,2018-01-03,NaN,-4.7,-0.4,-7.1,31.0,19.0,42.3,8.6,88.7,2.2,3.5,7156
3,2018-01-04,NaN,-4.7,-0.7,-8.7,39.0,24.0,43.0,6.2,63.9,1.4,3.5,7102
4,2018-01-05,NaN,-3.0,1.6,-5.6,51.0,35.0,48.4,8.2,84.5,1.7,3.6,7705


# feature 추가

> 2018,2019,2020 년도에 공휴일&주말, 공휴일&평일, 공휴일&월요일, 공휴일&금요일 등 공휴일또한 예측에 영향이 있는 것으로 판단하여 
>
> 공휴일은 1 부여

In [ ]:
train['holiday']=0
train.loc[train['date']=='2018-01-01', 'holiday'] =1
train.loc[train['date']=='2018-02-15', 'holiday'] =1
train.loc[train['date']=='2018-02-16', 'holiday'] =1
train.loc[train['date']=='2018-02-17', 'holiday'] =1
train.loc[train['date']=='2018-03-01', 'holiday'] =1
train.loc[train['date']=='2018-05-05', 'holiday'] =1
train.loc[train['date']=='2018-05-07', 'holiday'] =1
train.loc[train['date']=='2018-05-22', 'holiday'] =1
train.loc[train['date']=='2018-06-06', 'holiday'] =1
train.loc[train['date']=='2018-06-13', 'holiday'] =1
train.loc[train['date']=='2018-08-15', 'holiday'] =1
train.loc[train['date']=='2018-09-23', 'holiday'] =1
train.loc[train['date']=='2018-09-24', 'holiday'] =1
train.loc[train['date']=='2018-09-25', 'holiday'] =1
train.loc[train['date']=='2018-09-26', 'holiday'] =1
train.loc[train['date']=='2018-10-03', 'holiday'] =1
train.loc[train['date']=='2018-10-09', 'holiday'] =1
train.loc[train['date']=='2018-12-25', 'holiday'] =1

train.loc[train['date']=='2019-01-01', 'holiday'] =1
train.loc[train['date']=='2019-02-04', 'holiday'] =1
train.loc[train['date']=='2019-02-05', 'holiday'] =1
train.loc[train['date']=='2019-02-06', 'holiday'] =1
train.loc[train['date']=='2019-03-01', 'holiday'] =1
train.loc[train['date']=='2019-05-12', 'holiday'] =1
train.loc[train['date']=='2019-06-06', 'holiday'] =1
train.loc[train['date']=='2019-08-15', 'holiday'] =1
train.loc[train['date']=='2019-09-12', 'holiday'] =1
train.loc[train['date']=='2019-09-13', 'holiday'] =1
train.loc[train['date']=='2019-09-14', 'holiday'] =1
train.loc[train['date']=='2019-10-03', 'holiday'] =1
train.loc[train['date']=='2019-10-09', 'holiday'] =1
train.loc[train['date']=='2019-12-25', 'holiday'] =1

train.loc[train['date']=='2020-01-01', 'holiday'] =1
train.loc[train['date']=='2020-01-24', 'holiday'] =1
train.loc[train['date']=='2020-01-25', 'holiday'] =1
train.loc[train['date']=='2020-01-26', 'holiday'] =1
train.loc[train['date']=='2020-01-27', 'holiday'] =1
train.loc[train['date']=='2020-03-01', 'holiday'] =1
train.loc[train['date']=='2020-04-15', 'holiday'] =1
train.loc[train['date']=='2020-04-30', 'holiday'] =1
train.loc[train['date']=='2020-05-05', 'holiday'] =1
train.loc[train['date']=='2020-06-06', 'holiday'] =1
train.loc[train['date']=='2020-08-15', 'holiday'] =1
train.loc[train['date']=='2020-08-17', 'holiday'] =1
train.loc[train['date']=='2020-09-30', 'holiday'] =1
train.loc[train['date']=='2020-10-01', 'holiday'] =1
train.loc[train['date']=='2020-10-02', 'holiday'] =1
train.loc[train['date']=='2020-10-03', 'holiday'] =1
train.loc[train['date']=='2020-10-09', 'holiday'] =1
train.loc[train['date']=='2020-12-25', 'holiday'] =1


test_df.loc[train['date']=='2021-01-01', 'holiday'] =1
test_df.loc[train['date']=='2021-02-11', 'holiday'] =1
test_df.loc[train['date']=='2021-02-12', 'holiday'] =1
test_df.loc[train['date']=='2021-02-13', 'holiday'] =1
test_df.loc[train['date']=='2021-03-01', 'holiday'] =1
test_df.loc[train['date']=='2021-05-05', 'holiday'] =1
test_df.loc[train['date']=='2021-05-19', 'holiday'] =1
test_df.loc[train['date']=='2021-06-06', 'holiday'] =1
test_df.loc[train['date']=='2021-08-15', 'holiday'] =1
test_df.loc[train['date']=='2021-08-16', 'holiday'] =1
test_df.loc[train['date']=='2021-09-20', 'holiday'] =1
test_df.loc[train['date']=='2021-09-21', 'holiday'] =1
test_df.loc[train['date']=='2021-09-22', 'holiday'] =1
test_df.loc[train['date']=='2021-10-03', 'holiday'] =1
test_df.loc[train['date']=='2021-10-04', 'holiday'] =1
test_df.loc[train['date']=='2021-10-09', 'holiday'] =1
test_df.loc[train['date']=='2021-10-11', 'holiday'] =1
test_df.loc[train['date']=='2021-12-25', 'holiday'] =1
test_df.loc[train['date']=='2021-12-27', 'holiday'] =1

# train data 함수 적용 & 결측값 처리
> 함수 적용 (1)
>> 
>> 위에서 생성한 함수 중 연/월/일 분리하는 함수 적용
>
> precipitaion, PM10, PM2.5 에 있는 결측값 처리
>
>> 강수량의 경우 결측값을 0, 미세먼지의 경우 전체(2019~2020) 미세먼지에 대해 평균치로 적용
> 
> 함수 적용 (2)
>>
>> train 데이터에 요일 부여 ( train_day_of_the_week)
>>
>> get_dummies 를 통하여 요일 데이터를 one-hot encoding 진행
> rental 숫자 자체가 큰 것을 확인하여 Log1p 를 통해 값을 줄여줌 (스케일 시켜줌) -> 나중에 예측 후 expm1 으로 스케일 풀어줌
>
> 함수 적용 (3)
>> 불쾌지수 (discomfort_index), 체감온도 (temp_sensible), 일교차 , 미세먼지 신호등 (미세먼지 지수에 따른 값 부여) 진행


In [ ]:
#결측값 처리-강수량, 미세먼지
train_=seperate_date(train)
train_['precipitation']=train_['precipitation'].fillna(0)
train_['PM10'] = train_['PM10'].fillna(train_['PM10'].mean())
train_['PM2.5'] = train_['PM2.5'].fillna(train_['PM2.5'].mean())
train_ = train_.fillna(train_.mean())
#연도별 rental rate 변화
rental_rate(train_)
#train 요일 부여
train_=train_day_of_the_week(train_)
# 요일 one-hot encoding
train_= pd.get_dummies(data = train_,columns = ['day_name'])
#rental log scaling
train_["rental"] = np.log1p(train_["rental"])
#불쾌지수
discomfort_index(train_)
#체감온도
temp_sensible(train_)
#일교차
train_.insert(5,'temp_diff',train_['temp_highest'] - train_['temp_lowest'])
#미세먼지 신호등
pm_trafficlight(train_)

# test data 함수 적용 & 결측값 처리
> 함수 적용 (1)
>> 
>> 위에서 생성한 함수 중 연/월/일 분리하는 함수 적용
>
> precipitaion, PM10, PM2.5 에 있는 결측값 처리
>
>> 강수량의 경우 결측값을 0, 미세먼지의 경우 전체(2019~2020) 미세먼지에 대해 평균치로 적용
> 
> 함수 적용 (2)
>>
>> test 데이터에 요일 부여 (test_day_of_the_week)
>>
>> get_dummies 를 통하여 요일 데이터를 one-hot encoding 진행
> rental 숫자 자체가 큰 것을 확인하여 Log1p 를 통해 값을 줄여줌 (스케일 시켜줌) -> 나중에 예측 후 expm1 으로 스케일 풀어줌
>
> 함수 적용 (3)
>> 불쾌지수 (discomfort_index), 체감온도 (temp_sensible), 일교차 , 미세먼지 신호등 (미세먼지 지수에 따른 값 부여) 진행

In [ ]:
#결측값 처리-강수량, 미세먼지
test_date=test_df['date']
test_df=seperate_date(test_df)
test_df['precipitation']=test_df['precipitation'].fillna(0)
test_df['PM10'] = test_df['PM10'].fillna(test_df['PM10'].mean())
test_df['PM2.5'] = test_df['PM2.5'].fillna(test_df['PM2.5'].mean())
#연도별 rental rate 변화
test_df=test_df.fillna(test_df.mean())
#train 요일 부여
test_df=test_day_of_the_week(test_df)
# 요일 one-hot encoding
test_df = pd.get_dummies(data = test_df,columns = ["day_name"])
#rental log scaling
discomfort_index(test_df)
#불쾌지수
temp_sensible(test_df)
#일교차
test_df.insert(5,'temp_diff',train_['temp_highest'] - train_['temp_lowest'])
#미세먼지 신호등
pm_trafficlight(test_df)

# 데이터 Scaling
> MinMax, Stadardization, Normalization, QuantileTransformer 등 다양한 scaling 기법을 사용하였으나, 
> QuantileTransformer() 가 제일 데이터 scaling 후 표현을 잘하여 선정
>> 'PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation' 의 경우 데이터를 시각화 하였을때, (x축->rental y축->앞에 표시) 제일 뭉쳐져 있다고 판단하여 해당 칼럼을 선택

In [ ]:
# 8등 코드
scaler1 = QuantileTransformer()
scaler2 = QuantileTransformer()
col___list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation']
scaler1.fit(train_[col___list])
X_train_scaled = scaler1.transform(train_[col___list])
train_[col___list] =X_train_scaled
scaler2.fit(test_df[col___list])
X_train_scaled = scaler2.transform(test_df[col___list])
test_df[col___list] =X_train_scaled

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2583: UserWarning: n_quantiles (1000) is greater than the total number of samples (365). n_quantiles is set to n_samples.
  warnings.warn(


# feature 추가 (train & test)
> correlation 을 통해 데이터를 보았을때, rental 과 상관관계가 제일 높은 feature 를 선택 후 조합하여 추가 feature 생성
>> dust = pm10 * pm2.5 & abs(일교차) & sunshine_rate/sunshine_sum & sunshine_sum 에 있는 결측값 채우기
>>
>> test 데이터의 경우 미세먼지를 통해 만든 신호등에서 4번케이스가 없기에 0으로 채워둠. 

In [ ]:
train_ = pd.get_dummies(data = train_,columns = ['pm'])
train_['PMs'] = train_['PM10'] * train_['PM2.5']
train_['absolute_diff'] = abs(train_['temp_highest'] - train_['temp_lowest'])
train_['rate/sum'] = train_['sunshine_rate'] / train_['sunshine_sum']
train_['rate/sum'] = train_['rate/sum'].fillna(method='bfill')


test_df = pd.get_dummies(data = test_df,columns = ['pm'])
test_df['pm_4']=0.0
test_df['pm_4']=test_df['pm_4'].astype('uint8')
test_df['PMs'] = test_df['PM10'] * test_df['PM2.5']
test_df['absolute_diff'] = abs(test_df['temp_highest'] - test_df['temp_lowest'])
test_df['rate/sum'] = test_df['sunshine_rate'] / test_df['sunshine_sum']
test_df['rate/sum'] = test_df['rate/sum'].fillna(method='bfill')

# feature drop
> 불필요하다고 판단한 sunshine_sum을 train, test 데이터에서 drop 
>> rental-sunshine_sum 데이터 Plot을 보면, rate 을 통해서 진행하는 것이 더 유의미하다고 판단. (실제로 성능차이가 sum 을 제외한 것이 더 좋았음 rate 을 제거한 것보다)

In [ ]:
train_=train_.drop(columns='sunshine_sum')
test_df=test_df.drop(columns='sunshine_sum')

# train, test 데이터 확인
> 사전에 진행한 전처리 과정이 제대로 되었는지 .info() 를 통해 확인

In [ ]:
train_.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   precipitation       1095 non-null   float64
 1   temp_mean           1095 non-null   float64
 2   temp_highest        1095 non-null   float64
 3   temp_lowest         1095 non-null   float64
 4   temp_sensible       1095 non-null   float64
 5   temp_diff           1095 non-null   float64
 6   PM10                1095 non-null   float64
 7   PM2.5               1095 non-null   float64
 8   humidity            1095 non-null   float64
 9   sunshine_rate       1095 non-null   float64
 10  wind_mean           1095 non-null   float64
 11  wind_max            1095 non-null   float64
 12  rental              1095 non-null   float64
 13  holiday             1095 non-null   int64  
 14  year                1095 non-null   int64  
 15  month               1095 non-null   int64  
 16  day   

# 모델 선택 후 예측
> 모델은 kaggle 에서 제일 많은 우승을 들고 가고 있는 xgboost 를 선택.

In [ ]:
import xgboost
from xgboost import XGBRegressor
from xgboost import XGBRegressor as model

y = train_['rental']
x= train_.drop('rental',axis =1)
model = XGBRegressor(objective='reg:squarederror',learning_rate=0.1,max_depth = 4, n_estimators = 1000)
model.fit(x, y)
pred = model.predict(test_df)      
pred = pd.DataFrame(pred, columns=['rental'])
pred = np.expm1(pred)*1.35

# 결과 도출
> submission 코드, rental 에 예측한 prediction 값 넣기

In [ ]:
sample['rental'] = pred
sample

,date,rental
0,2021-01-01,27550.500000
1,2021-01-02,26879.683594
2,2021-01-03,17775.929688
3,2021-01-04,33266.390625
4,2021-01-05,26452.082031
...,...,...
360,2021-12-27,34484.773438
361,2021-12-28,37353.417969
362,2021-12-29,46339.921875
363,2021-12-30,35921.660156


# to_csv

In [ ]:
sample.to_csv('7.02result2.csv',index=False, encoding = 'UTF-8')